In [3]:
#Import necessary libraries
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Input, Flatten, Dropout
from tensorflow.keras.metrics import Precision,Recall,BinaryAccuracy, F1Score

In [4]:
import numpy as np
import pandas as pd

In [5]:
import keras
keras.__version__

'3.2.1'

## Data Import and Cleaning

In [6]:
data_dir = "/kaggle/input/note-dataset/Image Classification"

In [ ]:
image_exts=['jpeg','jpg','bmp','png']

for image_class in os.listdir(data_dir):
  for image in os.listdir(os.path.join(data_dir,image_class)):
    image_path = os.path.join(data_dir ,image_class,image)
    try:
      img = cv2.imread(image_path)
      tip = imghdr.what(image_path)
      if tip not in image_exts:
        print('Image not in ext list {}'.format(image_path))
        os.remove(image_path)
    except:
      print('Issue with image {}'.format(image_path))

In [7]:
for i  in os.listdir(data_dir):
  print(len(os.listdir(os.path.join(data_dir,i))))

316
366


## Load Dataset

In [9]:
data=tf.keras.utils.image_dataset_from_directory(data_dir,batch_size=16)

Found 682 files belonging to 2 classes.


In [10]:
#Image resizing and Rescaling
IMG_SIZE = 256

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

#Data Augmentation
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomContrast(0.3, seed=None)
])

## Train Val Test Split

In [16]:
train_size = int(len(data)*0.7)
val_size = int(len(data)*0.2)
test_size = int(len(data)*0.1)+1
print(train_size, val_size, test_size )

30 8 5


In [10]:
train = data.take(train_size)
test = data.skip(train_size).take(test_size)
val = data.skip(train_size+test_size).take(val_size)

In [11]:
print(len(train))
print(len(test))
print(len(val))

30
5
8


## Resizing, Scaling and Augmentation

In [12]:
scale_ds = train.map(lambda x, y: (resize_and_rescale(x, training=True), y))
aug_ds = scale_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
train_ds = scale_ds.concatenate(aug_ds)

In [17]:
for x, y in train_ds:
    print("Input shape:", x.shape)
    print("Shape of batch:", x[0].shape)
    print("Label shape:", y.shape)
    break

Input shape: (16, 256, 256, 3)
Shape of batch: (256, 256, 3)
Label shape: (16,)


Converting Keras Dataset to Numpy arrays to use it for tuning

In [13]:
train_data_np = []
train_labels_np = []
for images, labels in train_ds:
    train_data_np.append(images.numpy())
    train_labels_np.append(labels.numpy())

In [14]:
train_data_np = np.concatenate(train_data_np)
train_labels_np = np.concatenate(train_labels_np)

In [15]:
print(len(train_data_np))
print(len(train_labels_np))

960
960


## Hyper Paramter Tuning

In [22]:
def model_builder(hp):
  model = tf.keras.Sequential()
  model.add(Input(shape=(256, 256, 3)))

  hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'softmax'])
  hp_layer_1 = hp.Choice('layer_1', values = [16,32,64])
  hp_layer_2 = hp.Choice('layer_2', values = [16,32,64])
  hp_layer_3 = hp.Choice('layer_3', values = [16,32,64])
  hp_learning_rate = hp.Choice('learning_rate', values=[0.001, 0.01, 0.1])

  model.add(Conv2D(hp_layer_1,(3,3),1,activation=hp_activation))
  model.add(Conv2D(hp_layer_2,(3,3),1,activation=hp_activation))
  model.add(MaxPooling2D())

  model.add(Conv2D(hp_layer_3,(3,3),1,activation=hp_activation))
  model.add(MaxPooling2D())

  model.add(Flatten())
  model.add(Dense(256,activation=hp_activation,))
  model.add(Dense(1,activation = "sigmoid"))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.losses.BinaryCrossentropy(),
                metrics=['accuracy'])
  return model

In [23]:
import keras_tuner as kt

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='/kaggle/working/dir',
                     project_name='x')

In [24]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [25]:
tuner.search(train_data_np, train_labels_np, epochs=10, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 00m 04s]

Best val_accuracy So Far: 0.8177083134651184
Total elapsed time: 00h 10m 05s


In [26]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

## Viewing Best Parameters

In [28]:
best_activation = best_hps.get('activation')
best_layer_1 = best_hps.get('layer_1')
best_layer_2 = best_hps.get('layer_2')
best_layer_3 = best_hps.get('layer_3')
best_learning_rate = best_hps.get('learning_rate')

print("Best Activation:", best_activation)
print("Best Layer 1:", best_layer_1)
print("Best Layer 2:", best_layer_2)
print("Best Layer 3:", best_layer_3)
print("Best Learning Rate:", best_learning_rate)

Best Activation: relu
Best Layer 1: 32
Best Layer 2: 32
Best Layer 3: 64
Best Learning Rate: 0.001


## Building model using the Best Parameters

In [16]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [17]:
model = tf.keras.Sequential()
model.add(Input(shape=(256, 256, 3)))

model.add(Conv2D(32,(3,3),1,activation="relu"))
model.add(Conv2D(32,(3,3),1,activation="relu"))
model.add(MaxPooling2D())

model.add(Conv2D(64,(3,3),1,activation="relu"))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(256,activation="relu",))
model.add(Dense(1,activation = "sigmoid"))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss=tf.losses.BinaryCrossentropy(),
            metrics=['accuracy'])

In [18]:
# model = tuner.hypermodel.build(best_hps)
history = model.fit(train_data_np, train_labels_np, epochs=20, validation_split=0.2,
                    callbacks=[stop_early])

Epoch 1/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 126s 5s/step - accuracy: 0.5017 - loss: 6.8586 - val_accuracy: 0.4167 - val_loss: 0.8239
Epoch 2/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 141s 5s/step - accuracy: 0.6392 - loss: 0.6886 - val_accuracy: 0.5729 - val_loss: 0.7146
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 123s 5s/step - accuracy: 0.6909 - loss: 0.5996 - val_accuracy: 0.7240 - val_loss: 0.6019
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 143s 5s/step - accuracy: 0.8480 - loss: 0.3640 - val_accuracy: 0.6823 - val_loss: 0.7116
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - accuracy: 0.9182 - loss: 0.2510 - val_accuracy: 0.7188 - val_loss: 0.8842
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - accuracy: 0.9354 - loss: 0.1597 - val_accuracy: 0.8073 - val_loss: 0.7066


In [19]:
test_ds = test.map(lambda x, y: (resize_and_rescale(x, training=True), y))
test_data_np = []
test_labels_np = []
for images, labels in test_ds:
    test_data_np.append(images.numpy())
    test_labels_np.append(labels.numpy())

In [22]:
model.save('/kaggle/working/models/NoteClassificationModel.h5')

In [25]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()
f1_score = F1Score(threshold=0.5)
for batch in test_ds.as_numpy_iterator():
  X,y = batch
  yhat = model.predict(X)
  pre.update_state(y,yhat)
  re.update_state(y,yhat)
  acc.update_state(y,yhat)
  y = y.reshape(-1, 1)
  f1_score.update_state(y,yhat)

print(f"Precision: {pre.result().numpy()} Recall: {re.result().numpy()} Accuracy: {acc.result().numpy()} F1Score: {f1_score.result().numpy()}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 943ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 787ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 722ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 645ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step
Precision: 0.875 Recall: 0.8571428656578064 Accuracy: 0.8374999761581421 F1Score: [0.8659793]


#### Got Accuracy 83%

## Using Kfold Cross Validation to find the average accuracy of the model

In [26]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=7, shuffle=True, random_state=42)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold_1 = [] #save accuracy from each fold

# Convert TensorFlow dataset to NumPy arrays
train_data_np = []
train_labels_np = []

for images, labels in data:
    train_data_np.append(images.numpy())
    train_labels_np.append(labels.numpy())

train_data_np = np.concatenate(train_data_np)
train_labels_np = np.concatenate(train_labels_np)

for train, test in cv.split(train_data_np, train_labels_np):

  train_X = train_data_np[train]
  test_X = train_data_np[test]
  # print(shape(test_X))
  # print(train.shape)
  # print(train)
  # print(test.shape)
  print('   ')
  print(f'Training for fold {fold_no} ...')
  m_1 = tf.keras.Sequential()
  m_1.add(Input(shape=(256, 256, 3)))

  m_1.add(Conv2D(32,(3,3),1,activation="relu"))
  m_1.add(Conv2D(32,(3,3),1,activation="relu"))
  m_1.add(MaxPooling2D())

  m_1.add(Conv2D(64,(3,3),1,activation="relu"))
  m_1.add(MaxPooling2D())

  m_1.add(Flatten())
  m_1.add(Dense(256,activation="relu",))
  m_1.add(Dense(1,activation = "sigmoid"))

  m_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=['accuracy'])

  callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
  hist = m_1.fit(train_X, train_labels_np[train], epochs=20, validation_data = val,callbacks=[callback])

  m_1.save('models/model_fold_'+str(fold_no)+'.h5')

  pre = Precision()
  re = Recall()
  acc = BinaryAccuracy()
  f1_score = F1Score(threshold=0.5)

  yhat = m_1.predict(test_X)
  print(train_labels_np[test].shape, yhat.shape)
  pre.update_state(train_labels_np[test],yhat)
  re.update_state(train_labels_np[test],yhat)
  acc.update_state(train_labels_np[test],yhat)
  labels = train_labels_np.reshape(-1, 1)
  f1_score.update_state(labels[test],yhat)
  print(f"Precision: {pre.result().numpy()} Recall: {re.result().numpy()} Accuracy: {acc.result().numpy()} F1Score: {f1_score.result().numpy()}")
  acc_per_fold_1.append({'Precision':{pre.result().numpy()},
                       'Recall': {re.result().numpy()},
                       'Accuracy': {acc.result().numpy()},
                       'F1Score': {f1_score.result().numpy()[0]}
                       })
  fold_no = fold_no + 1

   
Training for fold 1 ...
Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 105s 5s/step - accuracy: 0.5186 - loss: 1796.8199 - val_accuracy: 0.4754 - val_loss: 2.1979
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 101s 5s/step - accuracy: 0.6804 - loss: 0.8151 - val_accuracy: 0.8934 - val_loss: 0.2675
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 101s 5s/step - accuracy: 0.9194 - loss: 0.2226 - val_accuracy: 0.9836 - val_loss: 0.0979
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - accuracy: 0.9841 - loss: 0.0940 - val_accuracy: 0.9672 - val_loss: 0.1387
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - accuracy: 0.9979 - loss: 0.0325 - val_accuracy: 0.9754 - val_loss: 0.1250
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 146s 6s/step - accuracy: 1.0000 - loss: 0.0029 - val_accuracy: 0.9754 - val_loss: 0.1649
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 776ms/step
(98,) (98, 1)
Precision: 0.8571428656578064 Recall: 0.7058823704719543 Accuracy: 0.7857142686843872 F1Score: [0.7741935]
   
Training for fold 2 ...
Epoch 1/20
19/19 ━━

KeyboardInterrupt: 

In [34]:
metrics_df = pd.DataFrame(acc_per_fold_1)

In [39]:
metrics_df = pd.DataFrame(acc_per_fold_1)
sum_= 0
for i in metrics_df["Accuracy"]:
  sum_ += next(iter(i))
print("Average accuracy: ", sum_/7)

Average accuracy:  0.7341077753475734


In [41]:
sum_= 0
for i in metrics_df["F1Score"]:
  sum_ += next(iter(i))
print("Average F1Score: ", sum_/7)

Average F1Score:  0.7376484274864197


#### Average Accuracy is around 73%

## Tried increasing the data using augmentation and did Kfold Cross validation again

In [20]:
cv = KFold(n_splits=7, shuffle=True, random_state=42)

# K-fold Cross Validation model evaluation
fold_no = 1
metrics_per_fold = [] #save accuracy from each fold

# Convert TensorFlow dataset to NumPy arrays
train_data_np = []
train_labels_np = []

scale_data = data.map(lambda x, y: (resize_and_rescale(x, training=True), y))
aug_data = scale_data.map(lambda x, y: (data_augmentation(x, training=True), y))
aug2_data = aug_data.map(lambda x, y: (data_augmentation(x, training=True), y))
train_data = scale_data.concatenate(aug_data)
train1_data = train_data.concatenate(aug2_data)

train_size = int(len(data)*0.8)
val_size = int(len(data)*0.2)+1

train = data.take(train_size)
val = data.skip(train_size).take(val_size)

for images, labels in train:
    train_data_np.append(images.numpy())
    train_labels_np.append(labels.numpy())

train_data_np = np.concatenate(train_data_np)
train_labels_np = np.concatenate(train_labels_np)

for train, test in cv.split(train_data_np, train_labels_np):
  train_X = train_data_np[train]
  test_X = train_data_np[test]
  # print(shape(test_X))
  # print(train.shape)
  # print(train)
  # print(test.shape)
  print('   ')
  print(f'Training for fold {fold_no} ...')
  model_1 = tf.keras.Sequential()
  model_1.add(Input(shape=(256, 256, 3)))

  model_1.add(Conv2D(32,(3,3),1,activation="relu"))
  model_1.add(Conv2D(32,(3,3),1,activation="relu"))
  model_1.add(MaxPooling2D())

  model_1.add(Conv2D(64,(3,3),1,activation="relu"))
  model_1.add(MaxPooling2D())

  model_1.add(Flatten())
  model_1.add(Dense(256,activation="relu",))
  model_1.add(Dense(1,activation = "sigmoid"))

  model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=['accuracy'])

  callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
  hist = model_1.fit(train_X, train_labels_np[train], epochs=20, validation_data = val,callbacks=[callback])

  model_1.save('models/model_fold_'+str(fold_no)+'.h5')

  pre = Precision()
  re = Recall()
  acc = BinaryAccuracy()
  f1_score = F1Score(threshold=0.5)

  yhat = model_1.predict(test_X)
  print(train_labels_np[test].shape, yhat.shape)
  pre.update_state(train_labels_np[test],yhat)
  re.update_state(train_labels_np[test],yhat)
  acc.update_state(train_labels_np[test],yhat)
  labels = train_labels_np.reshape(-1, 1)
  f1_score.update_state(labels[test],yhat)
  print(f"Precision: {pre.result().numpy()} Recall: {re.result().numpy()} Accuracy: {acc.result().numpy()} F1Score: {f1_score.result().numpy()}")
  metrics_per_fold.append({'Precision':{pre.result().numpy()},
                       'Recall': {re.result().numpy()},
                       'Accuracy': {acc.result().numpy()},
                       'F1Score': {f1_score.result().numpy()[0]}
                       })
  fold_no = fold_no + 1

   
Training for fold 1 ...
Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 76s 5s/step - accuracy: 0.4869 - loss: 1251.0979 - val_accuracy: 0.7246 - val_loss: 8.6441
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 74s 5s/step - accuracy: 0.6891 - loss: 13.5915 - val_accuracy: 0.6522 - val_loss: 4.2381
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 75s 5s/step - accuracy: 0.7983 - loss: 1.8695 - val_accuracy: 0.9058 - val_loss: 0.8434
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 74s 5s/step - accuracy: 0.9343 - loss: 0.3288 - val_accuracy: 0.8913 - val_loss: 0.9281
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9594 - loss: 0.1364 - val_accuracy: 0.9348 - val_loss: 0.8432
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 74s 5s/step - accuracy: 0.9769 - loss: 0.0511 - val_accuracy: 0.9275 - val_loss: 1.0707
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9854 - loss: 0.0484 - val_accuracy: 0.9638 - val_loss: 0.7740
Epoch 8/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9902 - loss: 0.0339 - 

In [21]:
model_1.save('/kaggle/working/models/NoteClassificationModel_Aug.h5')

In [22]:
metrics_df_1 = pd.DataFrame(metrics_per_fold)
sum_= 0
for i in metrics_df_1["Accuracy"]:
  sum_ += next(iter(i))
print("Average accuracy: ", sum_/7)

Average accuracy:  0.8676085727555412


#### Average Accuracy increased to 86%

In [23]:
sum_= 0
for i in metrics_df_1["F1Score"]:
  sum_ += next(iter(i))
print("Average F1Score: ", sum_/7)

Average F1Score:  0.8708762526512146


In [24]:
sum_= 0
for i in metrics_df_1["Precision"]:
  sum_ += next(iter(i))
print("Average Precision: ", sum_/7)

Average accuracy:  0.8690572466169085


In [25]:
sum_= 0
for i in metrics_df_1["Recall"]:
  sum_ += next(iter(i))
print("Average Recall: ", sum_/7)

Average accuracy:  0.8807445594242641
